In [2]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
import json

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
llm = ChatGroq(temperature=0, model_name = "llama-3.1-8b-instant", groq_api_key = os.environ["GROQ_API_KEY"])

In [4]:
text = """New Yorker Josh Kerben used to keep a careful eye on his petrol purchases, trying to stretch out his money by only partially filling the tank.

These days, however, he has been filling it to the top.

The decision is an indication of the increased financial breathing room being felt by millions of Americans, as petrol prices drop to their lowest level for three years, helping to contain rising living costs.

US inflation - the rate at which prices increase - rose slightly to 2.7% last month, official figures showed. Despite the increase, prices are rising far more slowly than in June 2022, when Russia's invasion of Ukraine sparked turmoil in global oil markets and sent fuel prices soaring.

Though the progress has appeared in economic data for months, economic dissatisfaction has remained high, as price increases in other areas, such as housing, overshadowed the wider improvement.

The issue played a key role in the US election, helping Donald Trump win re-election to the White House last month.

Now, however, surveys suggest opinions about the economy are finally starting to brighten, driven in large part by increased confidence among supporters of Trump, Mr Kerben among them, after the former president's election victory.

"Compared to two months ago, I would say I feel more confident," the 36-year-old property manager said. "Hopefully Trump will do something."

Ironically, the improvement in sentiment is arriving just as progress containing prices has appeared to stall.

The 2.7% inflation rate for November had been expected.

But it was up from 2.6% in October, marking the highest rate since July.

Gas prices - though down 8.1% compared with 2023 - rose 0.6% from October, while grocery prices jumped 0.5% over the month.

Prices of used cars, household furnishings and medical care also climbed.

The situation has raised questions about how Trump will deliver on his promises to lower prices for Americans - and what the US central bank, which wants to see an inflation rate of about 2%, should do next.

"Inflation has been coming steadily back into focus in the US," said Lindsay James, investment strategist at Quilter Investors.

"This is due in part to the lack of progress that has been made over the last three months, but also because of concerns that higher US government spending plus the introduction of Trump's tariffs could create a more inflationary backdrop."

The US central bank lowered interest rates for the first time in more than four years in September, citing the progress stabilising prices.

Many analysts still expect officials to announce another cut to interest rates at their meeting this month, but they have warned that rates are likely to stay higher than previously expected next year unless price increases in areas outside of petrol start to ease more significantly.

That's a conclusion that will come as no surprise to Americans like Grier Bowen.

The 48-year-old was diagnosed with cancer in 2017 and now relies on disability payments from the government, which she said have not kept up as expenses shot up in recent years.

Though the situation has been helped by lower petrol prices, she said the savings were not enough to offset higher costs elsewhere.

"You may save here but you've got to now reallocate somewhere else," she said.

Ms Bowen said she thought Trump would try to shake things up but she has yet to be convinced he will make a difference for the better.

"That's yet to be determined," she said."""

In [6]:
documents = [Document(page_content=text)]

llm_transformer_filtered = LLMGraphTransformer(llm=llm)
graph_documents_filtered  = llm_transformer_filtered.convert_to_graph_documents(documents)

In [22]:
print (graph_documents_filtered[0].nodes)

[Node(id='Josh Kerben', type='Person', properties={}), Node(id='Donald Trump', type='Person', properties={}), Node(id='Lindsay James', type='Person', properties={}), Node(id='Grier Bowen', type='Person', properties={}), Node(id='Us', type='Country', properties={}), Node(id='Russia', type='Country', properties={}), Node(id='Ukraine', type='Country', properties={}), Node(id='New Yorker', type='Person', properties={}), Node(id='Property Manager', type='Job', properties={}), Node(id='Investment Strategist', type='Job', properties={}), Node(id='Petrol', type='Resource', properties={}), Node(id='Gas Prices', type='Price', properties={}), Node(id='Grocery Prices', type='Price', properties={}), Node(id='Used Cars', type='Product', properties={}), Node(id='Household Furnishings', type='Product', properties={}), Node(id='Medical Care', type='Service', properties={}), Node(id='Us Central Bank', type='Institution', properties={}), Node(id='Quilter Investors', type='Institution', properties={})]


In [8]:
graph = NetworkxEntityGraph()

for node in graph_documents_filtered[0].nodes:
    graph.add_node(node.id)
    
for edge in graph_documents_filtered[0].relationships:
    graph._graph.add_edge(edge.source.id, edge.target.id, relation = edge.type)
    
graph_rag = GraphQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
)

In [23]:
print(graph._graph.nodes)

['Josh Kerben', 'Donald Trump', 'Lindsay James', 'Grier Bowen', 'Us', 'Russia', 'Ukraine', 'New Yorker', 'Property Manager', 'Investment Strategist', 'Petrol', 'Gas Prices', 'Grocery Prices', 'Used Cars', 'Household Furnishings', 'Medical Care', 'Us Central Bank', 'Quilter Investors', 'Disability Payments', 'Interest Rates', 'Petrol Prices']


In [14]:
import pandas as pd
df = pd.DataFrame(columns=['node1', 'node2', 'relation'])

for edge in graph_documents_filtered[0].relationships:
    df = pd.concat([df, pd.DataFrame({'node1': [edge.source.id], 'node2': [edge.target.id], 'relation': [edge.type]})], ignore_index=True)
df

,node1,node2,relation
0,Josh Kerben,Petrol,USES
1,Josh Kerben,Donald Trump,SUPPORTS
2,Donald Trump,Us,LEADER
3,Donald Trump,Us Central Bank,INFLUENCES
4,Lindsay James,Us Central Bank,WORKS_FOR
5,Lindsay James,Quilter Investors,WORKS_FOR
6,Grier Bowen,Us,CITIZEN
7,Grier Bowen,Medical Care,USES
8,Grier Bowen,Disability Payments,RECEIVES
9,Us,Russia,HAS_CONFLICT_WITH


In [24]:
question = """how donald trum will influence inflation"""
graph_rag.run(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Donald Trump
Full Context:
Donald Trump LEADER Us
Donald Trump INFLUENCES Us Central Bank

> Finished chain.


"I don't have enough information to determine how Donald Trump will influence inflation. The given knowledge triplets only state that Donald Trump is a leader of the United States and influences the US Central Bank, but they do not provide any information about his policies or actions regarding inflation."